In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, DeviceStatsMonitor
from pytorch_lightning.profilers import AdvancedProfiler
from pytorch_lightning import Trainer
from torchmetrics.functional import accuracy, mean_squared_error
from torch.utils.data import DataLoader, TensorDataset, random_split


import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'  # otherwise plotting will cause kerner to die
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


import sys
import matplotlib.pyplot as plt

# see if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available")

# see if cuDNN is available
if torch.backends.cudnn.enabled:
    print("cuDNN is available")

# set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is set to:", device)

CUDA is available
cuDNN is available
Device is set to: cuda


# Preprocessing

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer

class HouseCategoryCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['HouseCategory'] = X.apply(lambda row: self.categorize_house(row), axis=1)
        return X

    @staticmethod
    def categorize_house(row):
        latest_year = max(row['YearBuilt'], row['YearRemodAdd'])
        current_year = 2010  # Use a fixed current year
        if latest_year >= current_year - 20:
            age_category = 'Recent'
        elif latest_year >= current_year - 50:
            age_category = 'Modern'
        else:
            age_category = 'Historic'
        house_style = 'Single' if row['HouseStyle'] in ['1Story', '1.5Fin', '1.5Unf'] else 'Multi-Storey'
        bldg_type = '1Fam' if row['BldgType'] == '1Fam' else ('Townhouse' if row['BldgType'] in ['TwnhsE', 'Twnhs'] else 'Multi-Family')
        return f"{age_category}_{house_style}_{bldg_type}"
    

class BinaryConverter(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for column in self.columns:
            X[column] = (X[column] > 0).astype(int)
        return X

class PorchCombiner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['ClosedPorch'] = X['EnclosedPorch'] + X['OpenPorchSF'] + X['3SsnPorch']
        return X
    
class ReplaceNaNwithNone(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for column in self.columns:
            X[column] = X[column].fillna('None')
        return X

class DropNullRows(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return X.dropna(subset=[self.column])
    
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.drop(self.columns, axis=1, errors='ignore', inplace=True)
        return X
    
class CustomKNNImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns, n_neighbors=5):
        self.columns = columns
        self.n_neighbors = n_neighbors
        self.imputer = KNNImputer(n_neighbors=self.n_neighbors)

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self

    def transform(self, X):
        X = X.copy()
        X.loc[:, self.columns] = self.imputer.transform(X[self.columns])
        return X
    
class CustomFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        #X['HouseAge'] = X['YrSold'] - X['YearBuilt']
        #X['YearsSinceRemodel'] = X['YrSold'] - X['YearRemodAdd']
        X['TotalSqFeet'] = X['1stFlrSF'] + X['2ndFlrSF'] + X['TotalBsmtSF']
        X['TotalBathrooms'] = X['BsmtFullBath'] + 0.5 * X['BsmtHalfBath'] + X['FullBath'] + 0.5 * X['HalfBath']
        X['AreaQuality'] = X['GrLivArea'] * X['OverallQual']
        return X

class DebugTransformer(BaseEstimator, TransformerMixin):
    """ A transformer for debugging the pipeline by printing the DataFrame shape and column names. """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print("DataFrame shape:", X.shape)
        print("Columns:", X.columns.tolist())
        return X

In [3]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

numerical_cols_to_scale = [
    'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 
    'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'ScreenPorch', 
    'MoSold', 'YrSold', 'ClosedPorch', #'HouseAge', 'YearsSinceRemodel',
    'TotalSqFeet', 'TotalBathrooms', 'AreaQuality', 'MiscVal','PoolArea',
    'LowQualFinSF', 'BsmtFinSF2'
]

no_scaling_cols = ['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig',
                    'LandSlope','Neighborhood','Condition1','Condition2','RoofStyle','RoofMatl',
                    'Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation',
                    'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
                    'Heating','HeatingQC','CentralAir','KitchenQual','Functional',
                    'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive',
                    'SaleType','SaleCondition','HouseCategory', 'SalePrice']

pipeline = Pipeline([
    ('feature_engineering', CustomFeatureEngineer()),
    ('drop_null_rows', DropNullRows(column='Electrical')),  
    ('create_house_category', HouseCategoryCreator()),
    ('replace_nan_with_none', ReplaceNaNwithNone(columns=[
        'GarageType', 'MasVnrType', 'FireplaceQu', 'GarageFinish', 'GarageQual', 
        'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'
    ])),
    ('convert_to_binary', BinaryConverter(columns=['LowQualFinSF', 'BsmtFinSF2', 'PoolArea', 'MiscVal'])),
    ('combine_porch', PorchCombiner()),
    ('drop_columns', DropColumns(columns=[
        'Id', 'MSSubClass', 'Electrical', 'HouseStyle', 'BldgType', 
        'YearBuilt', 'YearRemodAdd', 'Alley', 'PoolQC', 'Fence', 
        'MiscFeature','EnclosedPorch', 'OpenPorchSF', '3SsnPorch'
    ])),
    ('final_processing', ColumnTransformer([
        ('num', Pipeline([
            ('impute', KNNImputer(n_neighbors=5))
            #,('scale', StandardScaler())
        ]), numerical_cols_to_scale),
        ('pass', 'passthrough', no_scaling_cols)
    ], remainder='passthrough'))
])

In [4]:
# remove pans columns display limit
pd.set_option('display.max_columns', None)

data = pd.read_csv("data/train.csv")
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,

In [5]:

data_processed = pipeline.fit_transform(data)

numerical_features = data_processed[:, :-38]

categorical_features = data_processed[:, -38:-2]
features = np.concatenate((numerical_features, categorical_features), axis=1)
target_regr = data_processed[:, -1]
target_clas = data_processed[:, -2]

In [6]:
# first row of numerical features
print(numerical_features[0])

[65.0 8450.0 7.0 5.0 196.0 706.0 150.0 856.0 856.0 854.0 1710.0 1.0 0.0
 2.0 1.0 3.0 1.0 8.0 0.0 2003.0 2.0 548.0 0.0 0.0 2.0 2008.0 61.0 2566.0
 3.5 11970.0 0.0 0.0 0.0 0.0]


In [7]:
# print first full 5 rows of categorical features
print(categorical_features[0])

['RL' 'Pave' 'Reg' 'Lvl' 'AllPub' 'Inside' 'Gtl' 'CollgCr' 'Norm' 'Norm'
 'Gable' 'CompShg' 'VinylSd' 'VinylSd' 'BrkFace' 'Gd' 'TA' 'PConc' 'Gd'
 'TA' 'No' 'GLQ' 'Unf' 'GasA' 'Ex' 'Y' 'Gd' 'Typ' 'None' 'Attchd' 'RFn'
 'TA' 'TA' 'Y' 'WD' 'Normal']


## Managing datatypes

In [8]:
df = pd.DataFrame(numerical_features)
df = df.apply(pd.to_numeric, errors='coerce')
numerical_features = df.to_numpy(dtype=np.float32)
categorical_features_df = pd.DataFrame(categorical_features)

# Convert each column to categorical type and use category codes as integers
for column in categorical_features_df.columns:
    categorical_features_df[column] = pd.Categorical(categorical_features_df[column])
    categorical_features_df[column] = categorical_features_df[column].cat.codes
print("Converted Data:")
print(categorical_features_df.head())

from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()
target_clas = label_encoder.fit_transform(target_clas)
target_regr = target_regr.astype(np.float32)

# Convert back to a numpy array of type int
categorical_features = categorical_features_df.to_numpy(dtype=np.int64)
categorical_features


Converted Data:
   0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  \
0   3   1   3   3   0   4   0   5   2   2   1   1  12  13   1   2   4   2   2   
1   3   1   3   3   0   2   0  24   1   2   1   1   8   8   2   3   4   1   2   
2   3   1   0   3   0   4   0   5   2   2   1   1  12  13   1   2   4   2   2   
3   3   1   0   3   0   0   0   6   2   2   1   1  13  15   2   3   4   0   4   
4   3   1   0   3   0   2   0  15   2   2   1   1  12  13   1   2   4   2   2   

   19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  
0   4   3   2   6   1   0   1   2   6   3   1   2   5   5   2   8   4  
1   4   1   0   6   1   0   1   3   6   5   1   2   5   5   2   8   4  
2   4   2   2   6   1   0   1   2   6   5   1   2   5   5   2   8   4  
3   1   3   0   6   1   2   1   2   6   2   5   3   5   5   2   8   0  
4   4   0   2   6   1   0   1   2   6   5   1   2   5   5   2   8   4  


array([[3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 0, ..., 2, 8, 4],
       ...,
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4],
       [3, 1, 3, ..., 2, 8, 4]], dtype=int64)

In [9]:
print(features[:1])
print(target_regr[:5])
print(target_clas[:5])

[[65.0 8450.0 7.0 5.0 196.0 706.0 150.0 856.0 856.0 854.0 1710.0 1.0 0.0
  2.0 1.0 3.0 1.0 8.0 0.0 2003.0 2.0 548.0 0.0 0.0 2.0 2008.0 61.0 2566.0
  3.5 11970.0 0.0 0.0 0.0 0.0 'RL' 'Pave' 'Reg' 'Lvl' 'AllPub' 'Inside'
  'Gtl' 'CollgCr' 'Norm' 'Norm' 'Gable' 'CompShg' 'VinylSd' 'VinylSd'
  'BrkFace' 'Gd' 'TA' 'PConc' 'Gd' 'TA' 'No' 'GLQ' 'Unf' 'GasA' 'Ex' 'Y'
  'Gd' 'Typ' 'None' 'Attchd' 'RFn' 'TA' 'TA' 'Y' 'WD' 'Normal']]
[208500. 181500. 223500. 140000. 250000.]
[10  7 10  4 10]


## Preparing embedding info

In [10]:
categorical_columns = data.select_dtypes(include=['object', 'category']).columns

# Calculate the number of unique values in each categorical column
embedding_info = []
for column in categorical_columns:
    categories = data[column].nunique()
    embedding_dim = min(50, categories // 2)    
    embedding_info.append({'column': column, 'categories': categories, 'embedding_dim': embedding_dim})

# Display the embedding info
embedding_info_df = pd.DataFrame(embedding_info)
print(embedding_info_df)

           column  categories  embedding_dim
0        MSZoning           5              2
1          Street           2              1
2           Alley           2              1
3        LotShape           4              2
4     LandContour           4              2
5       Utilities           2              1
6       LotConfig           5              2
7       LandSlope           3              1
8    Neighborhood          25             12
9      Condition1           9              4
10     Condition2           8              4
11       BldgType           5              2
12     HouseStyle           8              4
13      RoofStyle           6              3
14       RoofMatl           8              4
15    Exterior1st          15              7
16    Exterior2nd          16              8
17     MasVnrType           3              1
18      ExterQual           4              2
19      ExterCond           5              2
20     Foundation           6              3
21       B

## Modeling

In [11]:
class HousingNN(pl.LightningModule):
    def __init__(
        self,
        cat_cols,
        num_cols,
        num_layers,
        num_embds,
        embedding_dim,
        hidden_dim,
        output_dim_price,
        output_dim_category,
        price_loss_weight=1e-09,  # Best weight to get a good balance of tasks (this is hard to tune as optuna minimizes validation loss and tuning the weights will always end up favoring one task over the other)
        category_loss_weight=1,
        lr=0.001,
        optimizer_type="Adam",
        relu_type='LeakyReLU',
        dropoutrate=0.3,
    ):
        super().__init__()
        self.save_hyperparameters()

        # Learning rate and optimizer
        self.lr = lr
        self.optimizer_type = optimizer_type

        # Dropout rate
        self.dropoutrate = dropoutrate

        # numerical and categorical columns
        self.cat_cols = cat_cols
        self.num_cols = num_cols

        # Neural network architecture 
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.output_dim_price = output_dim_price
        self.output_dim_category = output_dim_category
        
        # Embedding layers
        self.num_embds = num_embds
        self.embedding_dim = embedding_dim
        self.total_embeddings = len(cat_cols) * embedding_dim
        self.embeddings = nn.ModuleList(
            [nn.Embedding(num_embds[i], embedding_dim) for i in range(len(cat_cols))]
        )
        print("Number of embedding layers:", len(self.embeddings))
        total_input_size = self.num_cols + self.total_embeddings

        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(total_input_size, hidden_dim),
            getattr(nn, relu_type)(),
            nn.Linear(hidden_dim, hidden_dim * 2),
        )

        # Add additional hidden layers
        for _ in range(num_layers - 1):
            self.fc.add_module("Linear_Layer_{}".format(_), nn.Linear(hidden_dim * 2, hidden_dim * 2))
            self.fc.add_module(f"{relu_type}_{_}", getattr(nn, relu_type)())

        # Output layers for Class inference
        self.output_category = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim_category),
        )

        # Output layers for Price inference
        self.output_price = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            getattr(nn, relu_type)(),
            nn.Linear(hidden_dim, output_dim_price),
        )

        # Loss weights
        self.price_loss_weight = price_loss_weight
        self.category_loss_weight = category_loss_weight

        # Initialize weights using kaiming uniform
        self.initialize_weights()

    def initialize_weights(self):
        for emb in self.embeddings:
            nn.init.kaiming_uniform_(emb.weight.data, nonlinearity='leaky_relu')
        for module in list(self.fc.children()) + list(self.output_price.children()) + list(self.output_category.children()):
            if isinstance(module, nn.Linear):
                nn.init.kaiming_uniform_(module.weight.data, nonlinearity='leaky_relu')

    def forward(self, cat_features, num_features):
    
        # Pass categorical features through embedding layers
        embedded_features = [emb(cat_features[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded_features = torch.cat(embedded_features, dim=1)
        
        # Concatenate embedded categorical features and numerical features
        concat_features = torch.cat([embedded_features, num_features], dim=1)
        shared_features = self.fc(concat_features)

        # Generate outputs for both tasks
        price_output = self.output_price(shared_features)
        category_output = self.output_category(shared_features)

        return price_output, category_output

    def training_step(self, batch, batch_idx):
        num_features, cat_features, price_targets, category_targets = batch
        price_output, category_output = self(cat_features, num_features)
        price_loss = nn.MSELoss()(price_output, price_targets)
        category_loss = nn.CrossEntropyLoss()(category_output, category_targets)

        loss = (self.price_loss_weight * price_loss +
                    self.category_loss_weight * category_loss)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        num_features, cat_features, price_targets, category_targets = batch
        price_output, category_output = self(cat_features, num_features)
        price_loss = nn.MSELoss()(price_output, price_targets)
        category_loss = nn.CrossEntropyLoss()(category_output, category_targets)

        loss = (self.price_loss_weight * price_loss +
                    self.category_loss_weight * category_loss)

        val_accuracy = accuracy(torch.argmax(category_output, dim=1), category_targets, task='multiclass', num_classes = len(np.unique(target_clas)))
        val_rmse = torch.sqrt(mean_squared_error(price_output.squeeze(), price_targets))

        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_price_loss', price_loss, on_epoch=True, prog_bar=True)
        self.log('val_category_loss', category_loss, on_epoch=True, prog_bar=True)
        self.log('val_accuracy', val_accuracy, on_epoch=True, prog_bar=True)
        self.log('val_rmse', val_rmse, on_epoch=True, prog_bar=True)

        return loss
    
    
    
    def test_step(self, batch, batch_idx):
        num_features, cat_features, price_targets, category_targets = batch
        price_output, category_output = self(cat_features, num_features)
        price_loss = nn.MSELoss()(price_output, price_targets)
        category_loss = nn.CrossEntropyLoss()(category_output, category_targets)

        test_accuracy = accuracy(torch.argmax(category_output, dim=1), category_targets, task='multiclass', num_classes = len(np.unique(target_clas)))
        test_rmse = torch.sqrt(mean_squared_error(price_output.squeeze(), price_targets))

        self.log('test_price_loss', price_loss)
        self.log('test_category_loss', category_loss)
        self.log('test_accuracy', test_accuracy)
        self.log('test_rmse', test_rmse)

        return {'test_price_loss': price_loss, 'test_category_loss': category_loss, 'test_accuracy': test_accuracy, 'test_rmse': test_rmse}

    def configure_optimizers(self):
        optimizer_class = getattr(torch.optim, self.optimizer_type)
        optimizer = optimizer_class(self.parameters(), lr=self.lr)
        return optimizer

In [12]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the numerical features
scaled_numerical_features = scaler.fit_transform(numerical_features)

num_features_tensor = torch.tensor(numerical_features, dtype=torch.float32)
cat_features_tensor = torch.tensor(categorical_features, dtype=torch.long)
target_regr_tensor = torch.tensor(target_regr, dtype=torch.float32)
target_clas_tensor = torch.tensor(target_clas, dtype=torch.long)  

dataset = TensorDataset(num_features_tensor, cat_features_tensor, target_regr_tensor, target_clas_tensor)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Initialize the scaler
scaler = StandardScaler()

# Extract numerical features for scaling from datasets
train_numerical_features = torch.stack([x[0] for x in train_dataset])
val_numerical_features = torch.stack([x[0] for x in val_dataset])
test_numerical_features = torch.stack([x[0] for x in test_dataset])

# Fit the scaler only on training data
scaler.fit(train_numerical_features.numpy())

# Transform each set using the fitted scaler
scaled_train_numerical_features = torch.tensor(scaler.transform(train_numerical_features), dtype=torch.float32)
scaled_val_numerical_features = torch.tensor(scaler.transform(val_numerical_features), dtype=torch.float32)
scaled_test_numerical_features = torch.tensor(scaler.transform(test_numerical_features), dtype=torch.float32)

# Rebuild the datasets using the scaled numerical features
train_dataset = TensorDataset(scaled_train_numerical_features, torch.stack([x[1] for x in train_dataset]), torch.stack([x[2] for x in train_dataset]), torch.stack([x[3] for x in train_dataset]))
val_dataset = TensorDataset(scaled_val_numerical_features, torch.stack([x[1] for x in val_dataset]), torch.stack([x[2] for x in val_dataset]), torch.stack([x[3] for x in val_dataset]))
test_dataset = TensorDataset(scaled_test_numerical_features, torch.stack([x[1] for x in test_dataset]), torch.stack([x[2] for x in test_dataset]), torch.stack([x[3] for x in test_dataset]))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(np.unique(target_clas_tensor.numpy()))
num_categorical_cols = cat_features_tensor.shape[1]  
num_numerical_features = num_features_tensor.shape[1] 
num_embds = [len(np.unique(cat_features_tensor[:, i])) for i in range(num_categorical_cols)]

print("Number of classes:", num_classes)
print("Number of categorical columns:", num_categorical_cols)
print("Number of numerical features:", num_numerical_features)
print("Number of unique categories per column:", num_embds)

Number of classes: 16
Number of categorical columns: 36
Number of numerical features: 34
Number of unique categories per column: [5, 2, 4, 4, 2, 5, 3, 25, 9, 8, 6, 8, 15, 16, 4, 4, 5, 6, 5, 5, 5, 7, 7, 6, 5, 2, 4, 7, 6, 7, 4, 6, 6, 3, 9, 6]


In [13]:


model = HousingNN(
    cat_cols=[i for i in range(num_categorical_cols)],  
    num_cols=num_numerical_features,  
    num_layers=5, 
    num_embds=num_embds, 
    embedding_dim=10,  
    hidden_dim=128,
    output_dim_price=1,
    output_dim_category=num_classes,
    
)

# Setup trainer with optional callbacks for checkpointing and early stopping
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=True,
    mode='min',
    min_delta= 0.0001
)

trainer = pl.Trainer(
    max_epochs=50,
    callbacks=[checkpoint_callback, early_stopping],
    accelerator='gpu',  
    devices=1 if torch.cuda.is_available() else 0, 
)

# Fit the model
trainer.fit(model, train_loader, val_loader)

Number of embedding layers: 36


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generat

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (16) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Epoch 14: 100%|██████████| 16/16 [00:00<00:00, 29.28it/s, v_num=45, train_loss_step=11.80, val_loss=8.710, val_price_loss=6.03e+9, val_category_loss=2.670, val_accuracy=0.482, val_rmse=7.61e+4, train_loss_epoch=7.490]

Metric val_loss improved by 1.261 >= min_delta = 0.0001. New best score: 8.706


Epoch 19: 100%|██████████| 16/16 [00:00<00:00, 27.66it/s, v_num=45, train_loss_step=5.340, val_loss=10.60, val_price_loss=6.24e+9, val_category_loss=4.320, val_accuracy=0.261, val_rmse=7.77e+4, train_loss_epoch=7.330] 

Monitored metric val_loss did not improve in the last 5 records. Best score: 8.706. Signaling Trainer to stop.


Epoch 19: 100%|██████████| 16/16 [00:00<00:00, 25.38it/s, v_num=45, train_loss_step=5.340, val_loss=10.60, val_price_loss=6.24e+9, val_category_loss=4.320, val_accuracy=0.261, val_rmse=7.77e+4, train_loss_epoch=7.330]


In [14]:
trainer.test(dataloaders=test_loader)

c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at C:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\checkpoints\model-epoch=14-val_loss=8.71.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at C:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\checkpoints\model-epoch=14-val_loss=8.71.ckpt
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `

Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 65.75it/s]

c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 54.94it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.44090908765792847
   test_category_loss       2.6582489013671875
     test_price_loss           4592699904.0
        test_rmse              67954.3828125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_price_loss': 4592699904.0,
  'test_category_loss': 2.6582489013671875,
  'test_accuracy': 0.44090908765792847,
  'test_rmse': 67954.3828125}]

In [15]:
import optuna
from optuna.integration import PyTorchLightningPruningCallback

def objective(trial: optuna.Trial):
    # Define hyperparameters using trial object
    
    num_layers = trial.suggest_int("num_layers", 1, 5)
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256, 512, 1024])
    embedding_dim = trial.suggest_int("embedding_dim", 5, 20)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer_type = trial.suggest_categorical("optimizer_type", ["Adam", "SGD"])
    relu_type = trial.suggest_categorical("relu_type", ["LeakyReLU", "ReLU"])
    dropoutrate = trial.suggest_float("droupoutrate", 0, 0.5)
    
    # Create the model instance with the suggested hyperparameters
    model = HousingNN(
        cat_cols=[i for i in range(num_categorical_cols)],
        num_cols=num_numerical_features,
        num_layers=num_layers,
        num_embds=num_embds,
        embedding_dim=embedding_dim, 
        hidden_dim=hidden_dim,
        output_dim_price=1,
        output_dim_category=num_classes,
        price_loss_weight=1e-9,
        category_loss_weight=1,       # could also be tuned
        lr = lr,
        optimizer_type= optimizer_type,
        relu_type= relu_type,
        dropoutrate= dropoutrate
        
    )

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=True,
        mode='min',
        min_delta= 0.0001
        
    )
    
    # Set up PyTorch Lightning trainer
    trainer = pl.Trainer(
        logger=False,
        max_epochs=40,  # Shorter for tuning, longer for final training
        accelerator='gpu' if torch.cuda.is_available() else None,
        devices=1 if torch.cuda.is_available() else 0,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss"),
                   early_stopping]
    )
    
    # Fit model
    trainer.fit(model, train_loader, val_loader)

    # Retrieve the best validation accuracy and RMSE from logged metrics
    val_accuracy = trainer.callback_metrics.get('val_accuracy', 0)
    val_rmse = trainer.callback_metrics.get('val_rmse', float('inf'))

    # Optionally store these metrics in trial (useful for later analysis)
    trial.set_user_attr('val_accuracy', val_accuracy)
    trial.set_user_attr('val_rmse', val_rmse)
    
    # Return the best validation loss
    return trainer.callback_metrics["val_loss"].item()


c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Create a study object
study = optuna.create_study(direction="minimize")

# Optimize the study
study.optimize(objective, n_trials=200)  # Specify the number of trials you want

# After the optimization, you can print the best parameters
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2024-04-28 23:46:20,440] A new study created in memory with name: no-name-26b53c11-954b-4019-8245-90a3ab4f3ad5


Number of embedding layers: 36


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:653: Checkpoint directory c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params
-----------------------------------------------
0 | embeddings      | ModuleList | 1.4 K 
1 | fc              | Sequential | 653 K 
2 | output_category | Sequential | 534 K 
3 | output_price    | Sequential | 525 K 
-----------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.858     Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 66.84it/s]

c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\.venv\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch 0: 100%|██████████| 16/16 [00:00<00:00, 30.73it/s, train_loss_step=18.60, val_loss=12.70, val_price_loss=9.03e+9, val_category_loss=3.720, val_accuracy=0.239, val_rmse=7.9e+4, train_loss_epoch=30.40]

Metric val_loss improved. New best score: 12.744


Epoch 1: 100%|██████████| 16/16 [00:00<00:00, 32.92it/s, train_loss_step=6.530, val_loss=12.70, val_price_loss=9.03e+9, val_category_loss=3.720, val_accuracy=0.239, val_rmse=7.9e+4, train_loss_epoch=30.40]

[I 2024-04-29 00:00:21,359] Trial 199 pruned. Trial was pruned at epoch 1.


Best trial:
  Value: 6.860494136810303
  Params: 
    num_layers: 1
    hidden_dim: 512
    embedding_dim: 14
    lr: 0.028039151980518314
    optimizer_type: Adam
    relu_type: LeakyReLU
    droupoutrate: 0.4030157350955987


In [17]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Best trial:")
print(f"    Value: {trial.value}")
print("    Parameters: ")
for key, value in trial.params.items():
    print(f"      {key}: {value}")


best_trial = study.best_trial
best_accuracy = best_trial.user_attrs.get('val_accuracy', 'Metric not recorded')
best_rmse = best_trial.user_attrs.get('val_rmse', 'Metric not recorded')

print(f"  Best Validation Accuracy: {best_accuracy}")
print(f"  Best Validation RMSE: {best_rmse}")

Study statistics: 
  Number of finished trials:  200
  Best trial:
    Value: 6.860494136810303
    Parameters: 
      num_layers: 1
      hidden_dim: 512
      embedding_dim: 14
      lr: 0.028039151980518314
      optimizer_type: Adam
      relu_type: LeakyReLU
      droupoutrate: 0.4030157350955987
  Best Validation Accuracy: 0.7293577790260315
  Best Validation RMSE: 75607.171875


In [24]:
model = HousingNN(
    cat_cols=[i for i in range(num_categorical_cols)],  
    num_cols=num_numerical_features,  
    num_layers=1, 
    num_embds=num_embds, 
    embedding_dim=14,  
    hidden_dim=512,
    output_dim_price=1,
    output_dim_category=num_classes,
    lr=0.028039151980518314,
    relu_type='LeakyReLU',
    dropoutrate=0.4030157350955987
    
)

# Setup trainer with optional callbacks for checkpointing and early stopping
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints/',
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=3,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    verbose=True,
    mode='min',
    min_delta= 0.0001
)

trainer = pl.Trainer(
    max_epochs=1000,
    callbacks=[checkpoint_callback, early_stopping],
    accelerator='gpu',  
    devices=1 if torch.cuda.is_available() else 0, 
)

# Fit the model
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params
-----------------------------------------------
0 | embeddings      | ModuleList | 3.2 K 
1 | fc              | Sequential | 801 K 
2 | output_category | Sequential | 534 K 
3 | output_price    | Sequential | 525 K 
-----------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.455     Total estimated model params size (MB)


Number of embedding layers: 36
Epoch 0: 100%|██████████| 16/16 [00:00<00:00, 24.99it/s, v_num=49, train_loss_step=16.30, val_loss=14.10, val_price_loss=9.38e+9, val_category_loss=4.710, val_accuracy=0.344, val_rmse=9.47e+4, train_loss_epoch=33.70]

Metric val_loss improved. New best score: 14.088


Epoch 1: 100%|██████████| 16/16 [00:00<00:00, 29.33it/s, v_num=49, train_loss_step=15.20, val_loss=10.30, val_price_loss=7.89e+9, val_category_loss=2.390, val_accuracy=0.431, val_rmse=9.08e+4, train_loss_epoch=11.00]

...

Epoch 12: 100%|██████████| 16/16 [00:00<00:00, 45.32it/s, v_num=49, train_loss_step=10.50, val_loss=7.110, val_price_loss=5.99e+9, val_category_loss=1.120, val_accuracy=0.711, val_rmse=7.28e+4, train_loss_epoch=7.510]

Metric val_loss improved by 0.069 >= min_delta = 0.0001. New best score: 7.111


Epoch 32: 100%|██████████| 16/16 [00:00<00:00, 30.94it/s, v_num=49, train_loss_step=11.00, val_loss=9.830, val_price_loss=7.35e+9, val_category_loss=2.480, val_accuracy=0.739, val_rmse=8.21e+4, train_loss_epoch=7.120]

Monitored metric val_loss did not improve in the last 20 records. Best score: 7.111. Signaling Trainer to stop.


Epoch 32: 100%|██████████| 16/16 [00:00<00:00, 30.53it/s, v_num=49, train_loss_step=11.00, val_loss=9.830, val_price_loss=7.35e+9, val_category_loss=2.480, val_accuracy=0.739, val_rmse=8.21e+4, train_loss_epoch=7.120]


In [25]:
trainer.test(dataloaders=test_loader)

Restoring states from the checkpoint path at C:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\checkpoints\model-epoch=12-val_loss=7.11.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at C:\Users\Owner\Documents\MMA\MULTI TASK\Final Project\checkpoints\model-epoch=12-val_loss=7.11.ckpt


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 66.85it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.7045454382896423
   test_category_loss       1.3563281297683716
     test_price_loss           4198658048.0
        test_rmse             61701.86328125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_price_loss': 4198658048.0,
  'test_category_loss': 1.3563281297683716,
  'test_accuracy': 0.7045454382896423,
  'test_rmse': 61701.86328125}]